In [ ]:
!pip install comet_ml
import comet_ml
from comet_ml import Experiment
from comet_ml.integration.pytorch import log_model
!pip install lightning
import lightning
from lightning.fabric import Fabric
import pandas as pd
import torch
import torch.nn as nn
from torch.nn import functional as F
from torch.utils.data import TensorDataset, DataLoader
import re
import os
import lzma
from tqdm import tqdm
import mmap
import random
import matplotlib.pyplot as plt
import numpy as np
from google.colab import files

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 586.4/586.4 kB 6.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 3.3/3.3 MB 18.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 54.5/54.5 kB 9.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 252.8/252.8 kB 19.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 137.9/137.9 kB 19.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 54.3/54.3 kB 6.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 512.2/512.2 kB 23.9 MB/s eta 0:00:00
  Attempting uninstall: websocket-client
    Found existing installation: websocket-client 1.6.4
    Uninstalling websocket-client-1.6.4:
      Successfully uninstalled websocket-client-1.6.4
  Attempting uninstall: python-box
    Found existing installation: python-box 7.1.1
    Uninstalling python-box-7.1.1:
      Successfully uninstalled python-box-7.1.1
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.0/2.0 MB 32.2 MB/s eta

In [ ]:
fabric = Fabric(precision="16-mixed")
device = fabric.device

INFO: Using 16-bit Automatic Mixed Precision (AMP)
INFO:lightning.pytorch.utilities.rank_zero:Using 16-bit Automatic Mixed Precision (AMP)


In [ ]:
from google.colab import drive
drive.mount('/content/drive/')

Mounted at /content/drive/


# Loading Jokes dataset

# Files loading

## mini-en

In [ ]:
%cd /content/drive/MyDrive/Project

/content/drive/MyDrive/Project


In [ ]:
# file_name = "mini-en.txt"
# text = ""
# with open(file_name, 'r', encoding='utf-8') as f:
#   text = f.read()

In [ ]:
# chars = sorted(list(set(text)))
# vocab_size = len(chars)
# print(''.join(chars))
# print(vocab_size)


## tiny stories

# Data Preprocessing

In [ ]:
def clean_text(text):
    # Lowercase the text
    text = text.lower()

    # Remove words that contain numbers
    text = re.sub(r'\w*\d\w*', '', text)

    # remove digits longer than 5
    text = re.sub(r'\d{5,}', '', text)

    # remove words longer than 20
    text = re.sub(r'\b\w{21,}\b', '', text)

    # Preserve meaningful punctuation and numbers
    text = re.sub(r"[^a-z0-9.,!?;'\s]", '', text)


    #normalize whitespaces
    text = re.sub(r'\s+', ' ', text)

    return text.strip()


In [ ]:
# # Redefining the clean_text function
# def clean_text(text):
#     # Lowercase the text
#     text = text.lower()
#     # print(text, ' p1')
#     # Remove words that contain numbers
#     text = re.sub(r'\w*\d\w*', '', text)

#     # remove digits longer than 5
#     text = re.sub(r'\d{5,}', '', text)

#     # remove words longer than 20
#     text = re.sub(r'\b\w{21,}\b', '', text)

#     # Preserve meaningful punctuation and numbers
#     text = re.sub(r"[^a-z0-9.,!?;\s']", '', text)

#     # Remove words with characters other than letters in them
#     text = re.sub(r'\s\w+[.,!?;]\w+\s', ' ', text)

#     words = text.split()
#     text = ' '.join(words)

#     # Truncate text to start after the first period and end at the last period
#     start = text.find('.') + 1
#     end = text.rfind('.')
#     if start != 0 and end != -1 and end > start:
#         text = text[start:end].strip()

#     text = re.sub(r'\,+[,\s]+[^\w]', ', ', text) # Remove occurences such as , ,
#     text = re.sub(r'\.+[\.\s]+[^\w]', '. ', text) # Remove occurences such as . .

#     text = re.sub(r'\s+\.', '.' , text) # If there is space before '.' remove it
#     text = re.sub(r'\s+[,]', ',' , text) # If there is space before ',' remove it
#     text = re.sub(r'\s+[!]', '!' , text) # If there is space before '!' remove it
#     text = re.sub(r'\s+[?]', '?' , text) # If there is space before '?' remove it

#     text = re.sub(r'http\S+', '', text) # Remove URLs
#     text = re.sub(r'\S+@\S+', '', text) # Remove email addresses
#     text = re.sub(r'.*;\s*$', '', text, flags=re.MULTILINE) # Remove lines that look like code (e.g., ending with semicolon)

#     # Remove unnecessary whitespaces and handle line breaks
#     text = text.strip()
#     text = re.sub(r'\s+', ' ', text)

#     # Remove repeating commas and periods
#     text = re.sub(r'[.]+', '.', text)
#     text = re.sub(r'[,]+', ',', text)
#     text = re.sub(r'\.+[,]+', ',', text)

#     return text

# def is_valid_line(line):
#   if line:

#     if len(line) < 2:
#       return False

#     if not re.search('[a-z]', line):
#       return False

#     return True

#   return False

In [ ]:
%cd /content/drive/MyDrive/Project/mini-en

/content/drive/.shortcut-targets-by-id/1DUoPwqF9OmnYJmuAwgDAFNsyXvHQQS9l/Project/mini-en


In [ ]:
# cleaned_file_name = "mini-en-cleaned-updated-fin.txt"
# with open(cleaned_file_name, 'r' , encoding='utf-8') as f:
#   chars = sorted(list(set(f.read())))
#   f.close()
# vocab_size = len(chars)
# print(''.join(chars))
# print(vocab_size)

In [ ]:
cleaned_lines = []

with open('/content/drive/MyDrive/Project/tiny-stories/TinyStories-train.txt', 'r') as f:
    for line in f:
        cleaned_line = clean_text(line)  # Replace 'clean_text' with your actual cleaning function
        cleaned_lines.append(cleaned_line)

cleaned_text2 = ''.join(cleaned_lines)

In [ ]:
chars = sorted(list(set(cleaned_text2)))
vocab_size = len(chars)
print(''.join(chars))
print(vocab_size)

 !',.;?abcdefghijklmnopqrstuvwxyz
33


In [ ]:
with open('vocab_tiny_stories.txt', 'w') as f:
    f.write(''.join(chars))

In [ ]:
#load tiny storie vocab into chars
with open('vocab_tiny_stories.txt', 'r') as f:
    chars = f.read()

FileNotFoundError: ignored

In [ ]:
chars = ''
with open("/content/drive/MyDrive/Project/Minipile/mini_pile_train_vocab.txt", 'r') as f:
  chars = f.read()
print("".join(chars))


 !',.;?abcdefghijklmnopqrstuvwxyz


In [ ]:
#to run with mini pile
#clean the test, train and validation mini-pile
%cd /content/drive/MyDrive/Minipile

# import os
# import re

# # Redefining the clean_text function
# def clean_text(text):
#     # Lowercase the text
#     text = text.lower()
#     # print(text, ' p1')
#     # Remove words that contain numbers
#     text = re.sub(r'\w*\d\w*', '', text)

#     # remove digits longer than 5
#     text = re.sub(r'\d{5,}', '', text)

#     # Preserve meaningful punctuation and numbers
#     text = re.sub(r"[^a-z0-9.,!?;\s']", '', text)

#     # Remove words with characters other than letters in them
#     text = re.sub(r'\s\w+[.,!?;]\w+\s', ' ', text)

#     words = text.split()
#     text = ' '.join(words)

#     # Truncate text to start after the first period and end at the last period
#     start = text.find('.') + 1
#     end = text.rfind('.')
#     if start != 0 and end != -1 and end > start:
#         text = text[start:end].strip()

#     text = re.sub(r'\,+[,\s]+[^\w]', ', ', text) # Remove occurences such as , ,
#     text = re.sub(r'\.+[\.\s]+[^\w]', '. ', text) # Remove occurences such as . .

#     text = re.sub(r'\s+\.', '.' , text) # If there is space before '.' remove it
#     text = re.sub(r'\s+[,]', ',' , text) # If there is space before ',' remove it
#     text = re.sub(r'\s+[!]', '!' , text) # If there is space before '!' remove it
#     text = re.sub(r'\s+[?]', '?' , text) # If there is space before '?' remove it
#     text = re.sub(r'http\S+', '', text) # Remove URLs
#     text = re.sub(r'\S+@\S+', '', text) # Remove email addresses
#     text = re.sub(r'.*;\s*$', '', text, flags=re.MULTILINE) # Remove lines that look like code (e.g., ending with semicolon)

#     # Remove unnecessary whitespaces and handle line breaks
#     text = text.strip()
#     text = re.sub(r'\s+', ' ', text)

#     # Remove repeating commas and periods
#     text = re.sub(r'[.]+', '.', text)
#     text = re.sub(r'[,]+', ',', text)
#     text = re.sub(r'\.+[,]+', ',', text)

#     return text

# def is_valid_line(line):
#   if line:

#     if len(line) < 2:
#       return False

#     if not re.search('[a-z]', line):
#       return False

#     return True

#   return False


# def process_chunk(chunk, out_file):
#     invalid_count = 0
#     for line in chunk:
#         line = clean_text(line)
#         if is_valid_line(line):
#             out_file.write(line + '\n')
#         else:
#             invalid_count += 1
#     return invalid_count


# file_name = "mini_pile_train.txt"
# chunk_size = 10000
# total_invalid_count = 0

# cleaned_file_name = "mini_pile_train_cleaned_v2.txt"
# with open(file_name, 'r', encoding='utf-8') as f, open(cleaned_file_name, 'w') as out_file:
#     while True:
#         lines = [f.readline() for _ in range(chunk_size)]
#         lines = [line for line in lines if line]  # Filter out empty lines

#         if not lines:
#             break  # Exit loop if no more lines to read

#         invalid_count = process_chunk(lines, out_file)
#         total_invalid_count += invalid_count

# print(f"Processed and cleaned the file. Total invalid lines skipped: {total_invalid_count}. Clean data written to {cleaned_file_name}")




In [ ]:
%cd /content/drive/MyDrive/Project/Minipile

In [ ]:
# file_name = "mini_pile_train_cleaned_v2.txt"
# text = ""
# with open(file_name, 'r', encoding='utf-8') as f:
#   text = f.read()

# chars = sorted(list(set(text)))
# vocab_size = len(chars)
# print(''.join(chars))
# print(vocab_size)
# #save the cleaned chars as file_name_vocab.txt
# with open('mini_pile_train_cleaned_v2_vocab.txt', 'w', encoding='utf-8') as f:
#   f.write(''.join(chars))



## vocab

In [ ]:
#loading the vocab associated with mini pile
#%cd /content/drive/MyDrive/Minipile
# %cd /content/drive/MyDrive/Project/Minipile
# vocab_file_name = "mini_pile_train_cleaned_v2_vocab.txt"
# with open(vocab_file_name, 'r' , encoding='utf-8') as f:
#   chars = sorted(list(set(f.read())))
#   f.close

# print(len(chars))
# print(''.join(chars))

/content/drive/MyDrive/Project/Minipile
34

 !',.;?abcdefghijklmnopqrstuvwxyz


# Tokenize

## Basic encoding

In [ ]:
stoi = { ch:i for i, ch in enumerate(chars)}
itos = { i:ch for i, ch in enumerate(chars)}

# encoder: string to int
encode = lambda s: [stoi[c] for c in s if c in stoi]

# decoder: int to string
decode = lambda l: ''.join([itos[i] for i in l if i in itos])

# Model

In [ ]:
# single head
class Head(nn.Module):

  def __init__(self, head_size):
    super().__init__()
    self.key = nn.Linear(n_embd, head_size, bias=False)
    self.query = nn.Linear(n_embd, head_size, bias=False)
    self.value = nn.Linear(n_embd, head_size, bias=False)
    self.register_buffer('tril', torch.tril(torch.ones(block_size, block_size)))

    self.dropout = nn.Dropout(dropout)

  def forward(self, x):
    B,T,C = x.shape
    k = self.key(x)
    q = self.query(x)

    wei = q @ k.transpose(-2, -1) * k.shape[-1]**-0.5
    wei = wei.masked_fill(self.tril[:T, :T] == 0, float('-inf'))
    wei = F.softmax(wei, dim=-1)
    wei = self.dropout(wei)

    v = self.value(x)
    out = wei @ v
    return out

# multi-head
class MultiHeadAttention(nn.Module):

  def __init__(self, num_heads, head_size):
    super().__init__()
    self.heads = nn.ModuleList([Head(head_size) for _ in range(num_heads)])
    self.proj = nn.Linear(n_embd, n_embd)
    self.dropout = nn.Dropout(dropout)

  def forward(self, x):
    out = torch.cat([h(x) for h in self.heads], dim=-1)
    out = self.dropout(self.proj(out))
    return out


class FeedForward(nn.Module):

  def __init__(self, n_embd):
    super().__init__()
    self.net = nn.Sequential(
        nn.Linear(n_embd, 4 * n_embd),
        nn.ReLU(),
        nn.Linear(4 * n_embd, n_embd),
        nn.Dropout(dropout),
    )

  def forward(self, x):
    return self.net(x)

class Block(nn.Module):

  def __init__(self, n_embd, n_head):
    super().__init__()
    head_size = n_embd // n_head
    self.sa = MultiHeadAttention(n_head, head_size)
    self.ffwd = FeedForward(n_embd)
    self.ln1 = nn.LayerNorm(n_embd)
    self.ln2 = nn.LayerNorm(n_embd)

  def forward(self, x):
    x = x + self.sa(self.ln1(x))
    x = x + self.ffwd(self.ln2(x))
    return x

  '''
  # post norm
  def forward(self, x):
    y = self.sa(x)
    x = self.ln1(x + y)
    y = self.ffwd(x)
    x = self.ln2(x + y)
    return x
  '''

In [ ]:
class GPTLanguageModel(nn.Module):
  def __init__(self):
    super().__init__()
    self.token_embedding_table = nn.Embedding(vocab_size, n_embd)
    self.position_embedding_table = nn.Embedding(block_size, n_embd)
    self.blocks = nn.Sequential(*[Block(n_embd, n_head=n_head) for _ in range(n_layer)])
    self.ln_f = nn.LayerNorm(n_embd)
    self.lm_head = nn.Linear(n_embd, vocab_size)

  def forward(self, idx, targets=None):
    B, T = idx.shape

    tok_emb = self.token_embedding_table(idx)
    pos_emb = self.position_embedding_table(torch.arange(T, device=device))
    x = tok_emb + pos_emb
    x = self.blocks(x)
    x = self.ln_f(x)
    logits = self.lm_head(x)

    if targets is None:
        loss = None
    else:
        B, T, C = logits.shape
        logits = logits.view(B*T, C)
        targets = targets.view(B*T)
        loss = F.cross_entropy(logits, targets)

    return logits, loss

  def generate(self, idx, max_new_tokens):

    for _ in range(max_new_tokens):
        idx_cond = idx[:, -block_size:]
        logits, loss = self(idx_cond)
        logits = logits[:, -1, :]
        probs = F.softmax(logits, dim=-1)
        idx_next = torch.multinomial(probs, num_samples=1)
        idx = torch.cat((idx, idx_next), dim=1)
    return idx


# Batches

drive  sample_data


In [ ]:
# memory map for using small snippets of text from a single file of any size
def get_random_chunk(split):
    filename = "TinyStories-train.txt" if split == 'train' else "TinyStories-valid.txt"
    with open(filename, 'rb') as f:
        with mmap.mmap(f.fileno(), 0, access=mmap.ACCESS_READ) as mm:
            # Determine the file size and a random position to start reading
            file_size = len(mm)
            start_pos = random.randint(0, (file_size) - block_size*batch_size)

            # Seek to the random position and read the block of text
            mm.seek(start_pos)
            block = mm.read(block_size*batch_size-1)

            # if the data is not long enough
            while len(block) - block_size <= 0:
              block = mm.read(block_size*batch_size-1)

            # Decode the block to a string, ignoring any invalid byte sequences
            decoded_block = block.decode('utf-8', errors='ignore').replace('\r', '')
            # clean the data
            # decoded_block = clean_text(decoded_block)
            # Do we need to clean the decoded block?

            # Train and test splits
            data = torch.tensor(encode(decoded_block), dtype=torch.long)

    return data

def get_batch(split):
    data = get_random_chunk(split)
    ix = torch.randint(len(data) - block_size, (batch_size,))
    x = torch.stack([data[i:i+block_size] for i in ix])
    y = torch.stack([data[i+1:i+block_size+1] for i in ix])
    x, y = x.to(device), y.to(device)
    return x, y


In [ ]:
# #use mini pile data - load and encode in batches
# #this blows up the CUDA RAM

# def data_generator(file_name, chunk_size=512):
#     with open(file_name, 'r', encoding='utf-8') as file:
#         while True:
#             data = file.read(chunk_size)
#             if not data:
#                 break
#             yield torch.tensor(encode(data), dtype=torch.long, device=device)

# # Usage
# generator = data_generator(cleaned_file_name, chunk_size=1024)  # Adjust chunk_size as needed

# # Initialize lists to store data
# train_data_list = []
# val_data_list = []

# # Process each chunk and accumulate
# for data_chunk in generator:
#     n = int(0.9 * len(data_chunk))  # Splitting each chunk into 90% train and 10% validation
#     train_data_list.append(data_chunk[:n])
#     val_data_list.append(data_chunk[n:])



# # Concatenate all chunks into single tensors
# train_data = torch.cat(train_data_list)
# val_data = torch.cat(val_data_list)



In [ ]:
# Train and test splits for mini en
# cleaned_file_name = "mini_pile_train_cleaned.txt"
# with open(cleaned_file_name, 'r', encoding='utf-8') as f:
#   data = torch.tensor(encode(f.read()), dtype=torch.long, device=device)
#   n = int(0.9*len(data)) # first 90% will be train, rest val
#   train_data = data[:n]
#   val_data = data[n:]

In [ ]:
# # data loading
# def get_batch(split):
#     # generate a small batch of data of inputs x and targets y
#     data = train_data if split == 'train' else val_data
#     ix = torch.randint(len(data) - block_size, (batch_size,))
#     x = torch.stack([data[i:i+block_size] for i in ix])
#     y = torch.stack([data[i+1:i+block_size+1] for i in ix])
#     x, y = x.to(device), y.to(device)
#     return x, y

In [ ]:
@torch.no_grad()
def estimate_loss():
    out = {}
    model.eval()
    for split in ['train', 'val']:
        losses = torch.zeros(eval_iters)
        for k in range(eval_iters):
            X, Y = get_batch(split)
            logits, loss = model(X, Y)
            losses[k] = loss.item()
        out[split] = losses.mean()
    model.train()
    return out

# Training

## Hyperparameters

In [ ]:
# hyperparameters
batch_size = 512
block_size = 128
max_iters = 1000
eval_interval = 200
learning_rate = 3e-5
eval_iters = 2000
n_embd = 200
n_head = 4
n_layer = 4
dropout = 0.0
#device = torch.device("cuda:0")
device = fabric.device
vocab_size = len(chars)

hyperparameters = {
    'batch_size': batch_size,
    'block_size': block_size,
    'max_iters': max_iters,
    'eval_interval': eval_interval,
    'learning_rate': learning_rate,
    'eval_iters': eval_iters,
    'n_embd': n_embd,
    'n_head': n_head,
    'n_layer': n_layer,
    'dropout': dropout,
}

# Hyperparameter -> Loss Visualization Data
# Essentially for every run, record the hyperparams and
# Plot them compared to old (save old in file)
vis_data = {
    'batch_size': batch_size,
    'block_size': block_size,
    'max_iters': max_iters,
    'eval_interval': eval_interval,
    'learning_rate': learning_rate,
    'eval_iters': eval_iters,
    'n_embd': n_embd,
    'n_head': n_head,
    'n_layer': n_layer,
    'dropout': dropout,
}

vis_data_file_name = "visual_data_during_training.txt"
vis_post_data_file_name = "visual_data_post_training.txt"

In [ ]:
# model
model = GPTLanguageModel()
# model.to(fabric.device)
model.to(device)
print(sum(p.numel() for p in model.parameters())/1e6, 'M parameters')

# Visualization Data
vis_data_iterations = [] # Format: [[iteration, train_loss, val_loss]]

# optimizer
optimizer = torch.optim.AdamW(model.parameters(), lr=learning_rate)

1.967233 M parameters


In [ ]:
torch.cuda.empty_cache()
import gc
gc.collect()

30

In [ ]:
'''# load the model and optimizer

model_384_8_8_path = '/content/drive/MyDrive/Project/Saved model/model_384_8_8.pth'
model_384_16_16_path = '/content/drive/MyDrive/Project/Saved model/model_384_16_16.pth'
model_128_4_4_path = '/content/drive/MyDrive/Project/Saved model/model_128_4_4.pth'

checkpoint = torch.load(model_384_8_8_path)
checkpoint = torch.load(model_384_16_16_path)
model.load_state_dict(checkpoint['model_state_dict'])
optimizer.load_state_dict(checkpoint['optimizer_state_dict'])
'''

"# load the model and optimizer\n\nmodel_384_8_8_path = '/content/drive/MyDrive/Project/Saved model/model_384_8_8.pth'\nmodel_384_16_16_path = '/content/drive/MyDrive/Project/Saved model/model_384_16_16.pth'\nmodel_128_4_4_path = '/content/drive/MyDrive/Project/Saved model/model_128_4_4.pth'\n\ncheckpoint = torch.load(model_384_8_8_path)\ncheckpoint = torch.load(model_384_16_16_path)\nmodel.load_state_dict(checkpoint['model_state_dict'])\noptimizer.load_state_dict(checkpoint['optimizer_state_dict'])\n"

In [ ]:
model_128_4_4_path = '/content/drive/MyDrive/Project/Saved Model/model_128_4_4.pth'
checkpoint = torch.load(model_128_4_4_path)
model.load_state_dict(checkpoint['model_state_dict'])
optimizer.load_state_dict(checkpoint['optimizer_state_dict'])

In [ ]:
%cd /content/drive/MyDrive/Project/tiny-stories

/content/drive/.shortcut-targets-by-id/10m65ZjTAPRJQ-QSuZpZdpGEB2CTwLxJi/tiny-stories


'/root'

In [ ]:
model_128_8_8_path = '/content/drive/MyDrive/Project/Saved Model/model_128_8_8_updated.pth'
checkpoint = torch.load(model_128_8_8_path)
model.load_state_dict(checkpoint['model_state_dict'])
optimizer.load_state_dict(checkpoint['optimizer_state_dict'])

In [ ]:
'''
fabric.launch()

experiment = Experiment(
  api_key="78u2AfbhkXeTChB3Kzb7FhOEY",
  project_name="JokeGPT",
  workspace="lzh0212"
)

experiment.log_parameters(hyperparameters)

for iter in range(max_iters):

  # other code...


  if iter % eval_interval == 0 or iter == max_iters - 1:
    losses = estimate_loss()
    experiment.log_metric("train_loss", losses['train'], epoch=iter)
    experiment.log_metric("val_loss", losses['val'], epoch=iter)
    print(f"step {iter}: train loss {losses['train']:.4f}, val loss {losses['val']:.4f}")


# after the training loop
experiment.end()
'''

In [ ]:
fabric.launch()

experiment = Experiment(
  api_key="8D1CVm6D2zRnIoB9RCQghv2V0",
  project_name="general",
  workspace="isfahanisasan"
)

experiment.log_parameters(hyperparameters)

for iter in range(max_iters):
  # if iter % eval_interval == 0 or iter == max_iters - 1:
    # losses = estimate_loss()
    # vis_data_iterations.append([iter, losses['train'], losses['val']])
    # print(f"step {iter}: train loss {losses['train']:.4f}, val loss {losses['val']:.4f}")

  xb, yb = get_batch('train')

  # evaluate
  logits, loss = model(xb, yb)
  if iter % eval_interval == 0 or iter == max_iters - 1:
    print(f"loss at {iter}: {loss:.4f}")
  optimizer.zero_grad(set_to_none=True)
  loss.backward()
  optimizer.step()

  if iter % eval_interval == 0 or iter == max_iters - 1:
    losses = estimate_loss()
    experiment.log_metric("train_loss", losses['train'], epoch=iter)
    experiment.log_metric("val_loss", losses['val'], epoch=iter)
    print(f"step {iter}: train loss {losses['train']:.4f}, val loss {losses['val']:.4f}")

log_model(experiment, model, model_name="Model 1")
experiment.end()

COMET WARNING: As you are running in a Jupyter environment, you will need to call `experiment.end()` when finished to ensure all metrics and code are logged before exiting.
COMET INFO: Couldn't find a Git repository in '/content/drive/.shortcut-targets-by-id/10m65ZjTAPRJQ-QSuZpZdpGEB2CTwLxJi/tiny-stories' nor in any parent directory. Set `COMET_GIT_DIRECTORY` if your Git Repository is elsewhere.
COMET INFO: Experiment is live on comet.com https://www.comet.com/isfahanisasan/general/27b28582c9b446638540022f4615f669



loss at 0: 1.9033
step 0: train loss 1.8666, val loss 1.8651
loss at 200: 1.8570
step 200: train loss 1.8180, val loss 1.8171
loss at 400: 1.7499
step 400: train loss 1.7719, val loss 1.7711
loss at 600: 1.7724
step 600: train loss 1.7335, val loss 1.7322
loss at 800: 1.6901


In [ ]:
log_model(experiment, model, model_name="Model 1")
experiment.end()

In [ ]:
losses = estimate_loss()
vis_data_iterations.append([max_iters-1, losses['train'].numpy(), losses['val'].numpy()])
print(f"train loss {losses['train']:.4f}, val loss {losses['val']:.4f}")

KeyboardInterrupt: ignored

# Visualization

In [ ]:
#Visualization -> graph performance based on different hyperparameters

#PRE-TRAINING
# Todo:
  # Write vis_data_iterations to file and plot it
  # Write vis_data to file and plot it
  # READ FROM FILES TO PLOT PAST DATA!

# Dont uncomment... vis_data_iterations = [[3,4,5]]

val_x_points = []
val_y_points = []
entries = []

# just added an extra open to guarantee that file is created
# if it does not exist...
with open(vis_data_file_name, 'a' , encoding='utf-8') as f:
  for entry in vis_data_iterations:
    entry_str = ' '.join(str(i) for i in entry) + '\n'

  f.write(entry_str)
  f.close()

with open(vis_data_file_name, 'r' , encoding='utf-8') as f:
  lines = f.read().split('\n')

  for line in lines:
    if not line:
      continue

    entries.append(line)

  entries = set(entries)

  for line in entries:
    line_entries = line.split(' ')
    val_x_points.append(int(line_entries[0]))
    val_y_points.append([float(i) for i in line_entries[1:]])

  f.close()

with open(vis_data_file_name, 'w' , encoding='utf-8') as f:
  f.write('\n'.join(entries) + '\n')
  f.close()

val_x_points = np.array(val_x_points)
val_y_points = np.array(val_y_points)

plt.plot(val_x_points, val_y_points)

plt.title("Losses over Iterations")
plt.xlabel("Iteration #")
plt.ylabel("Training and Validation Loss")

plt.show()

#JOKE TRAINING

ValueError: ignored

In [ ]:
vis_data["loss"] = estimate_loss()

# ADD THE FEATURES YOU WANT TO PLOT LOSS FOR, EACH ONE GENERATES GRAPH FOR LOSS!
hyper_plots = ['batch_size', 'block_size', 'max_iters', 'learning_rate', 'n_layer']
# NOTE: All other props have to remain constant for plot
# I.e. if plotting loss for "batch_size", everything else (e.g "max_iters")
# has to remain the same, else could not accurately plot. That is all the other
# props have to match the ones defined in the hyperparameters section.

# Essentially it will plot specified feature compared to loss when all the
# other features are equal to the entries in hyperparameters.

# Plot them compared to old (save old in file)

data_lists = {}

for plot_type in hyper_plots:
  data_lists[plot_type] = {
      'x_points' : [],
      'y_points' : [],
  }

def process_dict(entry):
  for plot_type in hyper_plots:
    valid_point = True

    for item in hyperparameters.items():
      if item[0] == plot_type:
        continue
      if item[1] != entry[item[0]]:
        valid_point = False
        break

    if valid_point:
      data_lists[plot_type]['x_points'].append(entry[plot_type])
      data_lists[plot_type]['y_points'].append(entry['loss'])



hyper_str = str(vis_data)

with open(vis_post_data_file_name, 'a' , encoding='utf-8') as f:
  f.write(hyper_str+'\n')
  f.close()

with open(vis_post_data_file_name, 'r' , encoding='utf-8') as f:
  entries = []
  lines = f.read().split('\n')

  for line in lines:
    if not line:
      continue
    entries.append(line)

  entries = set(entries)

  for line in entries:
    entry_dict = eval(line)
    process_dict(entry_dict)

  f.close()

with open(vis_post_data_file_name, 'w' , encoding='utf-8') as f:
  f.write('\n'.join(entries) + '\n')
  f.close()

for plot_type in hyper_plots:
  x_points = np.array(data_lists[plot_type]['x_points'])
  y_points = np.array(data_lists[plot_type]['y_points'])

  plt.plot(x_points, y_points)

  plt.title(f'Losses over {plot_type}')
  plt.xlabel(plot_type)
  plt.ylabel(f'{plot_type} to Loss')

  plt.show()

# Save the model

In [ ]:
def save(model, optimizer, hyperparameters, base_path='/content/drive/MyDrive/Project/Saved Model/'):
    n_embd = hyperparameters['n_embd']
    n_head = hyperparameters['n_head']
    n_layer = hyperparameters['n_layer']

    filename = f"model_{n_embd}_{n_head}_{n_layer}.pth"
    full_path = base_path + filename

    torch.save({
        'model_state_dict': model.state_dict(),
        'optimizer_state_dict': optimizer.state_dict(),
        'hyperparameters': hyperparameters
    }, full_path)

    print(f"Checkpoint saved to {full_path}")

In [ ]:
save(model, optimizer, hyperparameters)

Checkpoint saved to /content/drive/MyDrive/Project/Saved Model/model_200_4_4.pth


# Result

In [ ]:
prompt = 'hello'
context = torch.tensor(encode(prompt), dtype=torch.long, device=device)
generated_chars = decode(model.generate(context.unsqueeze(0), max_new_tokens=200)[0].tolist())
print(generated_chars)

helloilk how mand mjap joked her to to noing up. he cat apims put sframeed the! laih?tig her they the sald buleaghtep hougham. she githeed nomeed, borre.lon.ton dex he platy to we aid subottege trime ry. h


In [ ]:
context = torch.zeros((1, 1), dtype=torch.long, device=device)
print(decode(model.generate(context, max_new_tokens=200)[0].tolist()))

  and mayourn, it anke ta dabmy, stetten's sad hat ance i he sus plool her was. of pim on a shey to couch ass.the to soulland antee died dand and anded sorsid bicket pugs.the day weard lemit.et ple she
